# 導入

# pip install jupyter
# pip install pandas
# pip install lxml
# pip install tqdm
# pip install selenium
# pip install webdriver-manager

# インポート

In [1]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup # Htmlの細かい要素を取り出したい場合に使うモジュール
import re #正規表現を使いたい場合のモジュール
import time
from tqdm.notebook import tqdm # 実行進捗を可視化できるモジュール

# 【リメーク版】競馬予想で始める機械学習

# データ取得

## 目標  
- 2023年のレース結果のテーブルをスクレイピングにより取得して、Pythonで扱えるようにする

## ステップ  
1. カレンダーのページから開催日一覧を取得（スクレイピング）  
    1. 2023年1月であれば：https://race.netkeiba.com/top/calendar.html?year=2023&month=1  
2. 開催ページからレースid一覧を取得 
    1. 2023年1月5日の開催であれば：https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105 
3. レース結果ページからレース結果テーブル一覧を取得  

### （用語の解説）スクレイピングとは  
- プログラムによって、Webページから自動的にデータを取得すること

In [2]:
from urllib.request import urlopen

url = "https://db.netkeiba.com/race/202505021011/"
html = urlopen(url).read()

In [3]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>\xcd\xa5\xbd\xd9\xcc\xc6\xc7\xcf\xa1\xc32025\xc7\xaf5\xb7\xee25\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1\

In [4]:
import pandas as pd

pd.read_html(html)

[    着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人 気  \
 0     1    7   15     カムニャック  牝3  55  シュタル  2:25.7    NaN   14.3    4   
 1     2    1    1  アルマヴェローチェ  牝3  55  岩田望来  2:25.7    アタマ    3.4    2   
 2     3    7   13     タガノアビー  牝3  55  藤岡佑介  2:25.9  1.1/4   49.0   10   
 3     4    2    3    パラディレーヌ  牝3  55  丹内祐次  2:26.0     クビ   17.7    7   
 4     5    3    5   リンクスティップ  牝3  55  Ｍ．デム  2:26.2  1.1/4    3.7    3   
 5     6    6   11   ウィルサヴァイブ  牝3  55  団野大成  2:26.4  1.1/4  328.8   18   
 6     7    6   12  ブラウンラチェット  牝3  55   レーン  2:26.6  1.1/2   15.3    6   
 7     8    8   17  ケリフレッドアスク  牝3  55   ディー  2:26.7     クビ  188.7   14   
 8     9    5    9   エンブロイダリー  牝3  55  ルメール  2:26.7     クビ    3.3    1   
 9    10    8   18  エリカエクスプレス  牝3  55  戸崎圭太  2:26.8    3/4   14.5    5   
 10   11    5   10  タイセイプランセス  牝3  55   石橋脩  2:26.9     クビ   42.8    9   
 11   12    2    4     アイサンサン  牝3  55  北村宏司  2:27.0      1  253.7   15   
 12   13    3    6    ビップデイジー  牝3  55 

In [5]:
pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,7,15,カムニャック,牝3,55,シュタル,2:25.7,NaN,14.3,4,474(+4),[西] 友道康夫
1,2,1,1,アルマヴェローチェ,牝3,55,岩田望来,2:25.7,アタマ,3.4,2,490(-6),[西] 上村洋行
2,3,7,13,タガノアビー,牝3,55,藤岡佑介,2:25.9,1.1/4,49.0,10,492(-4),[西] 千田輝彦
3,4,2,3,パラディレーヌ,牝3,55,丹内祐次,2:26.0,クビ,17.7,7,498(0),[西] 千田輝彦
4,5,3,5,リンクスティップ,牝3,55,Ｍ．デム,2:26.2,1.1/4,3.7,3,472(0),[西] 西村真幸
5,6,6,11,ウィルサヴァイブ,牝3,55,団野大成,2:26.4,1.1/4,328.8,18,432(+2),[西] 須貝尚介
6,7,6,12,ブラウンラチェット,牝3,55,レーン,2:26.6,1.1/2,15.3,6,442(-2),[東] 手塚貴久
7,8,8,17,ケリフレッドアスク,牝3,55,ディー,2:26.7,クビ,188.7,14,444(+6),[西] 藤原英昭
8,9,5,9,エンブロイダリー,牝3,55,ルメール,2:26.7,クビ,3.3,1,482(0),[東] 森一誠
9,10,8,18,エリカエクスプレス,牝3,55,戸崎圭太,2:26.8,3/4,14.5,5,456(0),[西] 杉山晴紀


## 開催日一覧を取得

In [6]:
url = "https://race.netkeiba.com/top/calendar.html?year=2023&month=1"
html = urlopen(url).read()
html

b'<!DOCTYPE html>\n<html>\n<head>\n<meta charset="EUC-JP">\n<!-- block=common__meta_tag_common_race_schedule (d) -->\n<meta http-equiv="content-language" content="ja">\n<meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n<meta name="viewport" content="width=1000">\n<meta name="format-detection" content="telephone=no" />\n<title>\xb3\xab\xba\xc5\xc6\xfc\xc4\xf8 | \xa5\xec\xa1\xbc\xa5\xb9\xbe\xf0\xca\xf3(JRA) - netkeiba</title>\n<meta name="description" content="\xc3\xe6\xb1\xfb\xb6\xa5\xc7\xcf\xa1\xcaJRA\xa1\xcb\xb3\xab\xba\xc5\xa5\xec\xa1\xbc\xa5\xb9\xa4\xce\xb3\xab\xba\xc5\xc6\xfc\xc4\xf8\xa5\xda\xa1\xbc\xa5\xb8\xa4\xc7\xa4\xb9\xa1\xa3\xc3\xe6\xb1\xfb\xb6\xa5\xc7\xcf\xa1\xcaJRA\xa1\xcb\xb3\xab\xba\xc5\xa5\xec\xa1\xbc\xa5\xb9\xa4\xce\xbd\xd0\xc7\xcf\xc9\xbd\xa4\xe4\xba\xc7\xbf\xb7\xa5\xaa\xa5\xc3\xa5\xba\xa1\xa2\xa5\xec\xa1\xbc\xa5\xb9\xb7\xeb\xb2\xcc\xc2\xae\xca\xf3\xa1\xa2\xca\xa7\xcc\xe1\xbe\xf0\xca\xf3\xa4\xf2\xa4\xcf\xa4\xb8\xa4\xe1\xa1\xa2\xb6\xa5\xc7\xcf\xcd\xbd\xc1\xdb\xa4\

In [7]:
from bs4 import BeautifulSoup # Htmlの細かい要素を取り出したい場合に使うモジュール

soup = BeautifulSoup(html)
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<!-- block=common__meta_tag_common_race_schedule (d) -->
<meta content="ja" http-equiv="content-language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1000" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>開催日程 | レース情報(JRA) - netkeiba</title>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" name="description"/>
<meta content="競馬,keiba,開催日程,スケジュール,開催日,netkeiba,ネット競馬,netkeiba" name="keywords"/>
<meta content="https://www.netkeiba.com/style/netkeiba.ja/image/netkeiba_logo_keiba.png" name="thumbnail"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="開催日程 | レース情報(JRA) - netkeiba" property="og:title"/>
<meta content="https://race.netkeiba.com/top/calendar.html" property="og:url"/>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最

In [8]:
a = soup.find("table", class_="Calendar_Table").find("a") #第二引数指定しないと、デフォルトで最初に見つかった要素が返却される
a

<a href="../top/race_list.html?kaisai_date=20230105" target="_parent" tile="">
<div class="RaceKaisaiBox HaveData">
<p><span class="Day">5</span></p>
<p><span class="JyoName">中山</span><span class="JName">中山金杯</span></p>
<p><span class="JyoName">中京</span><span class="JName">京都金杯</span></p>
</div><!-- /. RaceKaisaiBox-->
</a>

In [9]:
a["href"]

'../top/race_list.html?kaisai_date=20230105'

In [10]:
import re #正規表現を使いたい場合のモジュール

re.findall(r"kaisai_date=(\d{8})", a["href"])[0]

'20230105'

In [11]:
a_list = soup.find("table", class_="Calendar_Table").find_all("a")

In [12]:
kaisai_data_list = []
for a in a_list:
    kaisai_data = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
    kaisai_data_list.append(kaisai_data)

In [13]:
kaisai_data_list

['20230105',
 '20230107',
 '20230108',
 '20230109',
 '20230114',
 '20230115',
 '20230121',
 '20230122',
 '20230128',
 '20230129']

In [14]:
from_ = "2023-01"
to_ = "2023-12"

In [15]:
for date in pd.date_range(from_, to_, freq="MS"):
    year = date.year
    month = date.month
    url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
    print(url)

https://race.netkeiba.com/top/calendar.html?year=2023&month=1
https://race.netkeiba.com/top/calendar.html?year=2023&month=2
https://race.netkeiba.com/top/calendar.html?year=2023&month=3
https://race.netkeiba.com/top/calendar.html?year=2023&month=4
https://race.netkeiba.com/top/calendar.html?year=2023&month=5
https://race.netkeiba.com/top/calendar.html?year=2023&month=6
https://race.netkeiba.com/top/calendar.html?year=2023&month=7
https://race.netkeiba.com/top/calendar.html?year=2023&month=8
https://race.netkeiba.com/top/calendar.html?year=2023&month=9
https://race.netkeiba.com/top/calendar.html?year=2023&month=10
https://race.netkeiba.com/top/calendar.html?year=2023&month=11
https://race.netkeiba.com/top/calendar.html?year=2023&month=12


In [16]:
import time
from tqdm.notebook import tqdm # 実行進捗を可視化できるモジュール

def scrape_kaisai_date(from_, to_):
    kaisai_data_list = []
    for date in tqdm(pd.date_range(from_, to_, freq="MS")):
        year = date.year
        month = date.month
        url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
        html = urlopen(url).read() # スクレイピング
        time.sleep(1) # スクレイピングをFor文使用する際、絶対忘れないように（パフォーマンス低下防ぎ）
        soup = BeautifulSoup(html)
        a_list = soup.find("table", class_="Calendar_Table").find_all("a")
        for a in a_list:
            kaisai_data = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
            kaisai_data_list.append(kaisai_data)
    return kaisai_data_list

In [17]:
# 例えば
scrape_kaisai_date("2023-01", "2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

['20230105',
 '20230107',
 '20230108',
 '20230109',
 '20230114',
 '20230115',
 '20230121',
 '20230122',
 '20230128',
 '20230129',
 '20230204',
 '20230205',
 '20230211',
 '20230212',
 '20230218',
 '20230219',
 '20230225',
 '20230226',
 '20230304',
 '20230305',
 '20230311',
 '20230312',
 '20230318',
 '20230319',
 '20230325',
 '20230326',
 '20230401',
 '20230402',
 '20230408',
 '20230409',
 '20230415',
 '20230416',
 '20230422',
 '20230423',
 '20230429',
 '20230430',
 '20230506',
 '20230507',
 '20230513',
 '20230514',
 '20230520',
 '20230521',
 '20230527',
 '20230528',
 '20230603',
 '20230604',
 '20230610',
 '20230611',
 '20230617',
 '20230618',
 '20230624',
 '20230625',
 '20230701',
 '20230702',
 '20230708',
 '20230709',
 '20230715',
 '20230716',
 '20230722',
 '20230723',
 '20230729',
 '20230730',
 '20230805',
 '20230806',
 '20230812',
 '20230813',
 '20230819',
 '20230820',
 '20230826',
 '20230827',
 '20230902',
 '20230903',
 '20230909',
 '20230910',
 '20230916',
 '20230917',
 '20230918',

## 開催ページからrace_id取得

In [18]:
url = "https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105"

In [19]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [20]:
html = urlopen(url)
soup = BeautifulSoup(html)

In [21]:
soup.find("div", class_="RaceList_Box")

BeautifulSoupはjavascriptのような動的に生成オブジェクトを取得できないので、ChromeDriverを使う

In [22]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

# 最新版のChromeをインストールし、インストール先のパスが返却される
driver_path = ChromeDriverManager().install()
driver_path

'C:\\Users\\saiku\\.wdm\\drivers\\chromedriver\\win64\\140.0.7339.185\\chromedriver-win32/chromedriver.exe'

In [23]:
# インストールした最新版のChromeブラウザを起動する
driver = webdriver.Chrome(service=Service(driver_path))

In [24]:
# 指定URLよりページを開く
driver.get(url)

In [25]:
from selenium.webdriver.common.by import By

li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
li_list

[<selenium.webdriver.remote.webelement.WebElement (session="cf1b07f92263151c49000a9b1fb9e361", element="f.E4485D7ACD941FE976435339FC87F34A.d.90DE013A70799AE93DF6E7A6A8C0B98E.e.37")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cf1b07f92263151c49000a9b1fb9e361", element="f.E4485D7ACD941FE976435339FC87F34A.d.90DE013A70799AE93DF6E7A6A8C0B98E.e.38")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cf1b07f92263151c49000a9b1fb9e361", element="f.E4485D7ACD941FE976435339FC87F34A.d.90DE013A70799AE93DF6E7A6A8C0B98E.e.39")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cf1b07f92263151c49000a9b1fb9e361", element="f.E4485D7ACD941FE976435339FC87F34A.d.90DE013A70799AE93DF6E7A6A8C0B98E.e.40")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cf1b07f92263151c49000a9b1fb9e361", element="f.E4485D7ACD941FE976435339FC87F34A.d.90DE013A70799AE93DF6E7A6A8C0B98E.e.41")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cf1b07f92263151c49000a9b1

In [26]:
li = li_list[0]
li

<selenium.webdriver.remote.webelement.WebElement (session="cf1b07f92263151c49000a9b1fb9e361", element="f.E4485D7ACD941FE976435339FC87F34A.d.90DE013A70799AE93DF6E7A6A8C0B98E.e.37")>

In [27]:
href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
href

'https://race.netkeiba.com/race/result.html?race_id=202306010101&rf=race_list'

In [28]:
import re

re.findall(r"race_id=(\d{12})", href)[0]

'202306010101'

In [29]:
race_id_list = []
for li in li_list:
    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
    race_id = re.findall(r"race_id=(\d{12})", href)[0]
    # print(race_id)
    race_id_list.append(race_id)

In [30]:
len(race_id_list)

24

In [31]:
# Chromeブラウザを閉じる
driver.quit()

### 関数化

In [32]:
import scraping

In [33]:
kaisai_data_list = scraping.scrape_kaisai_date("2023-01", "2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

In [34]:
import time
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
import traceback

def scrape_race_id_list(kaisai_data_list: list[str]):
    options = Options()
    # バックグラウンドで実行
    options.add_argument("--headless")
    # 最新版のChromeをインストールし、インストール先のパスが返却される
    driver_path = ChromeDriverManager().install()
    race_id_list = []
    
    # インストールした最新版のChromeブラウザを起動する
    with webdriver.Chrome(service=Service(driver_path), options=options) as driver:
        for kaisai_data in tqdm(kaisai_data_list):
            url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_data}"
            try:
                driver.get(url)
                time.sleep(1)
                li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
                for li in li_list:
                    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
                    race_id = re.findall(r"race_id=(\d{12})", href)[0]
                    race_id_list.append(race_id)
            except:
                print(f"stopped at {url}")
                print(traceback.format_exc())
                break
    return race_id_list
        

In [35]:
race_id_list = scrape_race_id_list(kaisai_data_list)

  0%|          | 0/107 [00:00<?, ?it/s]

stopped at https://race.netkeiba.com/top/race_list.html?kaisai_date=20230611
Traceback (most recent call last):
  File "C:\Users\saiku\AppData\Local\Temp\ipykernel_17636\1079480709.py", line 23, in scrape_race_id_list
    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\tools\Python\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 602, in find_element
    return self._execute(Command.FIND_CHILD_ELEMENT, {"using": by, "value": value})["value"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\tools\Python\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 573, in _execute
    return self._parent.execute(command, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\tools\Python\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 455, in execute
    response = cast(RemoteConnection, self.command_executor).execute(driver

### スクリプトのチェック

In [36]:
 # インポートモジュールを自動で同期化
import scraping
%load_ext autoreload

In [37]:
%autoreload

In [38]:
race_id_list = scrape_race_id_list(kaisai_data_list[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

In [39]:
race_id_list

['202306010101',
 '202306010102',
 '202306010103',
 '202306010104',
 '202306010105',
 '202306010106',
 '202306010107',
 '202306010108',
 '202306010109',
 '202306010110',
 '202306010111',
 '202306010112',
 '202307010101',
 '202307010102',
 '202307010103',
 '202307010104',
 '202307010105',
 '202307010106',
 '202307010107',
 '202307010108',
 '202307010109',
 '202307010110',
 '202307010111',
 '202307010112',
 '202306010201',
 '202306010202',
 '202306010203',
 '202306010204',
 '202306010205',
 '202306010206',
 '202306010207',
 '202306010208',
 '202306010209',
 '202306010210',
 '202306010211',
 '202306010212',
 '202307010201',
 '202307010202',
 '202307010203',
 '202307010204',
 '202307010205',
 '202307010206',
 '202307010207',
 '202307010208',
 '202307010209',
 '202307010210',
 '202307010211',
 '202307010212',
 '202306010301',
 '202306010302',
 '202306010303',
 '202306010304',
 '202306010305',
 '202306010306',
 '202306010307',
 '202306010308',
 '202306010309',
 '202306010310',
 '202306010311

In [40]:
from urllib.request import urlopen
url = "https://db.netkeiba.com/race/202505021011"
html = urlopen(url).read()

In [41]:
import pandas as pd

pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,7,15,カムニャック,牝3,55,シュタル,2:25.7,NaN,14.3,4,474(+4),[西] 友道康夫
1,2,1,1,アルマヴェローチェ,牝3,55,岩田望来,2:25.7,アタマ,3.4,2,490(-6),[西] 上村洋行
2,3,7,13,タガノアビー,牝3,55,藤岡佑介,2:25.9,1.1/4,49.0,10,492(-4),[西] 千田輝彦
3,4,2,3,パラディレーヌ,牝3,55,丹内祐次,2:26.0,クビ,17.7,7,498(0),[西] 千田輝彦
4,5,3,5,リンクスティップ,牝3,55,Ｍ．デム,2:26.2,1.1/4,3.7,3,472(0),[西] 西村真幸
5,6,6,11,ウィルサヴァイブ,牝3,55,団野大成,2:26.4,1.1/4,328.8,18,432(+2),[西] 須貝尚介
6,7,6,12,ブラウンラチェット,牝3,55,レーン,2:26.6,1.1/2,15.3,6,442(-2),[東] 手塚貴久
7,8,8,17,ケリフレッドアスク,牝3,55,ディー,2:26.7,クビ,188.7,14,444(+6),[西] 藤原英昭
8,9,5,9,エンブロイダリー,牝3,55,ルメール,2:26.7,クビ,3.3,1,482(0),[東] 森一誠
9,10,8,18,エリカエクスプレス,牝3,55,戸崎圭太,2:26.8,3/4,14.5,5,456(0),[西] 杉山晴紀


In [42]:
import pickle

with open("race_id_list.pickle", "rb") as f:
    race_id_list = pickle.load(f)

In [43]:
len(race_id_list)

3456

In [44]:
for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    html = urlopen(url).read()
    df = pd.read_html(html)[0]
    # print(url)

In [45]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>\xa5\xb8\xa5\xe5\xa5\xcb\xa5\xa2\xa5\xab\xa5\xc3\xa5\xd7\xa1\xc32023\xc7\xaf1\xb7\xee5\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\x

In [46]:
with open("../data/html/202506010101.bin", "wb") as f:
    f.write(html)

In [47]:
#環境の違いを防げるパス連結
from pathlib import Path

HTML_DIR = Path("..", "data", "html")
HTML_DIR

WindowsPath('../data/html')

In [48]:
HTML_DIR / "202506010101.bin"

WindowsPath('../data/html/202506010101.bin')

In [49]:
with open(HTML_DIR / "202506010101.bin", "wb") as f:
    f.write(html)

In [50]:
race_id_list[:10]

['202306010101',
 '202306010102',
 '202306010103',
 '202306010104',
 '202306010105',
 '202306010106',
 '202306010107',
 '202306010108',
 '202306010109',
 '202306010110']

In [51]:
import time

for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    html = urlopen(url).read()
    time.sleep(1)
    with open(HTML_DIR / f"{race_id}.bin", "wb") as f:
        f.write(html)


In [56]:
from tqdm.notebook import tqdm
def scrap_html_race(race_id_list: list[str], save_dir: Path) -> list[Path]:
    html_path_list = []
    for race_id in tqdm(race_id_list):
        filepath = save_dir / f"{race_id}.bin"
        # binファイルが既に存在する場合はスキップする
        if filepath.is_file():
            print(f"skipped:{race_id}")
            continue
        url = f"https://db.netkeiba.com/race/{race_id}"
        html = urlopen(url).read()
        time.sleep(1)
        with open(filepath, "wb") as f:
            f.write(html)
        html_path_list.append(filepath)
    return html_path_list

In [57]:
html_path_list = scrap_html_race(race_id_list[:14], save_dir=HTML_DIR)

  0%|          | 0/14 [00:00<?, ?it/s]

skipped:202306010101
skipped:202306010102
skipped:202306010103
skipped:202306010104
skipped:202306010105
skipped:202306010106
skipped:202306010107
skipped:202306010108
skipped:202306010109
skipped:202306010110
skipped:202306010111
skipped:202306010112


In [54]:
html_path_list

[WindowsPath('../data/html/202306010101.bin'),
 WindowsPath('../data/html/202306010102.bin'),
 WindowsPath('../data/html/202306010103.bin'),
 WindowsPath('../data/html/202306010104.bin'),
 WindowsPath('../data/html/202306010105.bin'),
 WindowsPath('../data/html/202306010106.bin'),
 WindowsPath('../data/html/202306010107.bin'),
 WindowsPath('../data/html/202306010108.bin'),
 WindowsPath('../data/html/202306010109.bin'),
 WindowsPath('../data/html/202306010110.bin'),
 WindowsPath('../data/html/202306010111.bin'),
 WindowsPath('../data/html/202306010112.bin')]

In [58]:
HTML_RACE_DIR = HTML_DIR / "race"
HTML_RACE_DIR

WindowsPath('../data/html/race')

In [ ]:
# globを使って、ファイルパス全件を取得する
html_paths_race = list(HTML_RACE_DIR.glob("*"))
html_paths_race

[WindowsPath('../data/html/race/202305010101.bin'),
 WindowsPath('../data/html/race/202305010102.bin'),
 WindowsPath('../data/html/race/202305010103.bin'),
 WindowsPath('../data/html/race/202305010104.bin'),
 WindowsPath('../data/html/race/202305010105.bin'),
 WindowsPath('../data/html/race/202305010106.bin'),
 WindowsPath('../data/html/race/202305010107.bin'),
 WindowsPath('../data/html/race/202305010108.bin'),
 WindowsPath('../data/html/race/202305010109.bin'),
 WindowsPath('../data/html/race/202305010110.bin'),
 WindowsPath('../data/html/race/202305010111.bin'),
 WindowsPath('../data/html/race/202305010112.bin'),
 WindowsPath('../data/html/race/202305010201.bin'),
 WindowsPath('../data/html/race/202305010202.bin'),
 WindowsPath('../data/html/race/202305010203.bin'),
 WindowsPath('../data/html/race/202305010204.bin'),
 WindowsPath('../data/html/race/202305010205.bin'),
 WindowsPath('../data/html/race/202305010206.bin'),
 WindowsPath('../data/html/race/202305010207.bin'),
 WindowsPath

In [61]:
len(html_paths_race)

300

In [77]:
dfs = {}
for html_path in html_paths_race[:3]:
    with open(html_path, "rb") as f:
        # stemを使って、ファイル名を取得する
        race_id = html_path.stem
        html = f.read()
        df = pd.read_html(html)[0]
        # indexを付与し、切れ目をわかりやすくする
        df.index = [race_id] * len(df)
        dfs[race_id] = df
# 自称型のデータを繋げて一つのテーブルする
concat_df = pd.concat(dfs.values())
concat_df.index.name = "race_id"

In [73]:
dfs['202305010101']

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
202305010101,1,7,14,メイデンタワー,牝3,54,横山武史,1:26.1,NaN,1.6,1,444(-4),[東] 稲垣幸雄
202305010101,2,5,10,レディアス,牝3,54,津村明秀,1:26.8,4,6.0,3,424(-4),[東] 鈴木慎太
202305010101,3,5,9,アンジュール,牝3,54,嶋田純次,1:27.1,2,92.7,10,418(0),[東] 佐藤吉勝
202305010101,4,8,15,マテリアルガール,牝3,54,内田博幸,1:27.3,3/4,13.6,4,490(+14),[東] 西田雄一
202305010101,5,2,3,オールマキシマム,牝3,52,小林脩斗,1:27.9,3.1/2,5.8,2,450(+2),[東] 奥平雅士
202305010101,6,4,8,スピリチュアル,牝3,54,武藤雅,1:28.0,1/2,25.7,7,444(-6),[東] 武藤善則
202305010101,7,6,11,マラマプア,牝3,53,永野猛蔵,1:28.1,1,15.2,5,482(+2),[東] 伊藤圭三
202305010101,8,2,4,フジジュンフェイス,牝3,54,杉原誠人,1:28.5,2.1/2,108.1,11,442(-4),[東] 伊藤大士
202305010101,9,4,7,テイエムガーネット,牝3,54,武士沢友,1:28.6,1/2,22.6,6,468(+4),[東] 石栗龍彦
202305010101,10,3,6,カノンチャン,牝3,52,横山琉人,1:29.0,2.1/2,134.4,12,446(-4),[東] 高柳瑞樹


In [74]:
['202305010101'] * 4

['202305010101', '202305010101', '202305010101', '202305010101']

In [75]:
['202305010101'] * len(dfs)

['202305010101', '202305010101', '202305010101']

In [ ]:
# 自称型のデータを繋げて一つのテーブルする
pd.concat(dfs.values())

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
202305010101,1,7,14,メイデンタワー,牝3,54,横山武史,1:26.1,NaN,1.6,1,444(-4),[東] 稲垣幸雄
202305010101,2,5,10,レディアス,牝3,54,津村明秀,1:26.8,4,6.0,3,424(-4),[東] 鈴木慎太
202305010101,3,5,9,アンジュール,牝3,54,嶋田純次,1:27.1,2,92.7,10,418(0),[東] 佐藤吉勝
202305010101,4,8,15,マテリアルガール,牝3,54,内田博幸,1:27.3,3/4,13.6,4,490(+14),[東] 西田雄一
202305010101,5,2,3,オールマキシマム,牝3,52,小林脩斗,1:27.9,3.1/2,5.8,2,450(+2),[東] 奥平雅士
202305010101,6,4,8,スピリチュアル,牝3,54,武藤雅,1:28.0,1/2,25.7,7,444(-6),[東] 武藤善則
202305010101,7,6,11,マラマプア,牝3,53,永野猛蔵,1:28.1,1,15.2,5,482(+2),[東] 伊藤圭三
202305010101,8,2,4,フジジュンフェイス,牝3,54,杉原誠人,1:28.5,2.1/2,108.1,11,442(-4),[東] 伊藤大士
202305010101,9,4,7,テイエムガーネット,牝3,54,武士沢友,1:28.6,1/2,22.6,6,468(+4),[東] 石栗龍彦
202305010101,10,3,6,カノンチャン,牝3,52,横山琉人,1:29.0,2.1/2,134.4,12,446(-4),[東] 高柳瑞樹


In [ ]:
from tqdm.notebook import tqdm
def create_results(html_path_list: list[Path]) -> pd.DataFrame:
    dfs = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, "rb") as f:
            # stemを使って、ファイル名を取得する
            race_id = html_path.stem
            html = f.read()
            df = pd.read_html(html)[0]
            # indexを付与し、切れ目をわかりやすくする
            df.index = [race_id] * len(df)
            dfs[race_id] = df
    # 自称型のデータを繋げて一つのテーブルする
    concat_df = pd.concat(dfs.values())
    concat_df.index.name = "race_id"
    return concat_df

In [82]:
results = create_results(html_path_list=html_path_list)

  0%|          | 0/300 [00:00<?, ?it/s]

In [83]:
results

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202305010101,1,7,14,メイデンタワー,牝3,54.0,横山武史,1:26.1,NaN,1.6,1.0,444(-4),[東] 稲垣幸雄
202305010101,2,5,10,レディアス,牝3,54.0,津村明秀,1:26.8,4,6.0,3.0,424(-4),[東] 鈴木慎太
202305010101,3,5,9,アンジュール,牝3,54.0,嶋田純次,1:27.1,2,92.7,10.0,418(0),[東] 佐藤吉勝
202305010101,4,8,15,マテリアルガール,牝3,54.0,内田博幸,1:27.3,3/4,13.6,4.0,490(+14),[東] 西田雄一
202305010101,5,2,3,オールマキシマム,牝3,52.0,小林脩斗,1:27.9,3.1/2,5.8,2.0,450(+2),[東] 奥平雅士
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310010512,14,5,10,スウィートメリナ,牝6,54.0,小林凌大,1:10.7,ハナ,108.4,18.0,450(+8),[西] 羽月友彦
202310010512,15,1,1,トーホウデュラン,牡4,58.0,西村淳也,1:11.0,2,4.9,1.0,488(+4),[西] 石坂公一
202310010512,16,2,3,ピンクセイラー,牝4,55.0,秋山稔樹,1:11.4,2,33.2,10.0,460(+14),[東] 小笠倫弘
